# Taller E-Commerce - Juan Sebastian Angel Perez

## Importar bibliotecas

In [30]:
import pandas as pd
import numpy as np
import io # Usado para cargar datos de ejemplo desde un string
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento y Pipelines
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Métricas
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, accuracy_score, f1_score, roc_auc_score

# Configuraciones
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)

## Carga y Preparación de Datos (E-Commerce 2008)

In [31]:
# Cargar datos de e-commerce
path = "data/e-shop clothing 2008.csv"
df = pd.read_csv(path, sep=";")

# --- Limpieza de datos básica ---
# Estandarización de nombres de columnas
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('(', '')
    .str.replace(')', '')
)

# Transformar la columna page_2_clothing_model en variable categórica
df['page_2_clothing_model'] = df['page_2_clothing_model'].astype('category')

# Unificación de year, month, day en una sola columna de fecha
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])


## Paso 1: Feature Engineering y Creación del Target

### Definir el Target (y) y Evitar Data Leakage:

In [32]:
# Crear la variable objetivo (Visited_Sale_Page)

visited_sale = (
    df.groupby('session_id')['page_1_main_category']
    .apply(lambda x: 1 if 4 in x.values else 0)
    .reset_index(name='Visited_Sale_Page')
)

# Evitar Data Leakage

df_filtered = df[df['page_1_main_category'] != 4]



### Crear las Features (X) y Agregar a Nivel de Sesión

In [43]:
# 1. Crear features agregadas a nivel sesión

session_features = (
    df_filtered
    .groupby('session_id')
    .agg(
        total_clicks=('order', 'count'),
        max_price_seen=('price', 'max'),
        avg_price_seen=('price', 'mean'),
        countries=('country', 'nunique'),
        distinct_products=('page_2_clothing_model', 'nunique'),
        main_category_mode=('page_1_main_category', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    )
    .reset_index()
)

# 2. Crear el dataset final a nivel sesión uniendo features + target

df_sesiones = session_features.merge(visited_sale, on='session_id', how='left')

df_sesiones.head()

,session_id,total_clicks,max_price_seen,avg_price_seen,countries,distinct_products,main_category_mode,Visited_Sale_Page
0,1,7,57,41.857143,1,7,2,1
1,2,7,67,53.000000,1,5,2,1
2,3,5,48,42.000000,1,5,3,1
3,4,4,62,45.250000,1,4,1,0
4,5,1,57,57.000000,1,1,3,0


### ¡Pregunta Clave! Ya que tienes tu dataset final... ¿Cómo es la distribución de `Visited_Sale_Page`? ¿Es un problema balanceado o desbalanceado?

De a cuerdo a los reultados de *df_sesiones['Visited_Sale_Page'].value_counts(normalize=True) * 100* los datos están moderadamente desbalanceados.

In [48]:
df_sesiones['Visited_Sale_Page'].value_counts(normalize=True) * 100

Visited_Sale_Page
0    66.567941
1    33.432059
Name: proportion, dtype: float64

## Paso 2: El Pipeline de Preprocesamiento

In [50]:
# 1. Definir X e y
X = df_sesiones.drop(columns=['Visited_Sale_Page', 'session_id'])
y = df_sesiones['Visited_Sale_Page']

# 2. Listas de variables numéricas y categóricas
numeric_features = [
    'total_clicks',
    'max_price_seen',
    'avg_price_seen',
    'countries',
    'distinct_products'
]

categorical_features = [
    'main_category_mode'
]

# 3. Transformador para variables numéricas:
#    - Imputación (media)
#    - Escalado (StandardScaler)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# 4. Transformador para variables categóricas:
#    - Imputación (valor más frecuente)
#    - OneHotEncoder
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 5. ColumnTransformer que aplica cada bloque a sus columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


## Paso 3: "Jugando" con los Modelos

### Modelo 1 (Línea Base 1): `LogisticRegression`.

In [51]:
# 1. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 2. Pipeline con preprocessor + modelo
logreg_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# 3. Entrenar
logreg_model.fit(X_train, y_train)

# 4. Predicciones
y_pred = logreg_model.predict(X_test)
y_prob = logreg_model.predict_proba(X_test)[:, 1]

# 5. Resultados
logreg_results = {
    'accuracy': accuracy_score(y_test, y_pred),
    'f1': f1_score(y_test, y_pred, pos_label=1),
    'roc_auc': roc_auc_score(y_test, y_prob),
    'precision': precision_score(y_test, y_pred, pos_label=1),
    'recall': recall_score(y_test, y_pred, pos_label=1)
}

logreg_results

{'accuracy': 0.6784325456177086,
 'f1': 0.22382671480144403,
 'roc_auc': 0.6539236337811125,
 'precision': 0.5794392523364486,
 'recall': 0.13870246085011187}

### Modelo 2 (Línea Base 2): `DecisionTreeClassifier`

In [52]:
tree_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(
        random_state=42
    ))
])

# Entrenar
tree_model.fit(X_train, y_train)

# Predicciones
y_pred_tree = tree_model.predict(X_test)
y_prob_tree = tree_model.predict_proba(X_test)[:, 1]

# Métricas
tree_results = {
    'accuracy': accuracy_score(y_test, y_pred_tree),
    'f1': f1_score(y_test, y_pred_tree, pos_label=1),
    'roc_auc': roc_auc_score(y_test, y_prob_tree),
    'precision': precision_score(y_test, y_pred_tree, pos_label=1),
    'recall': recall_score(y_test, y_pred_tree, pos_label=1)
}

tree_results

{'accuracy': 0.6540532455877954,
 'f1': 0.3587468810645966,
 'roc_auc': 0.5424014511481471,
 'precision': 0.4715743440233236,
 'recall': 0.28948545861297537}

### Modelo 3 (Bagging): `RandomForestClassifier`

In [53]:
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        class_weight='balanced'
    ))
])

# Entrenar
rf_model.fit(X_train, y_train)

# Predicciones
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

# Métricas
rf_results = {
    'accuracy': accuracy_score(y_test, y_pred_rf),
    'f1': f1_score(y_test, y_pred_rf, pos_label=1),
    'roc_auc': roc_auc_score(y_test, y_prob_rf),
    'precision': precision_score(y_test, y_pred_rf, pos_label=1),
    'recall': recall_score(y_test, y_pred_rf, pos_label=1)
}

rf_results

{'accuracy': 0.6214478013760095,
 'f1': 0.40656506447831187,
 'roc_auc': 0.5750983741933668,
 'precision': 0.4270935960591133,
 'recall': 0.38791946308724834}

### Modelo 4 (Boosting): `XGBClassifier` 

In [54]:
# Calcular scale_pos_weight = (negativos / positivos)
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale_pos_weight = neg / pos
scale_pos_weight

np.float64(1.9909875359539788)

In [55]:
xgb_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        random_state=42,
        eval_metric='logloss',
        scale_pos_weight=scale_pos_weight,
        n_estimators=300,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

# Entrenar
xgb_model.fit(X_train, y_train)

# Predicciones
y_pred_xgb = xgb_model.predict(X_test)
y_prob_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Métricas
xgb_results = {
    'accuracy': accuracy_score(y_test, y_pred_xgb),
    'f1': f1_score(y_test, y_pred_xgb, pos_label=1),
    'roc_auc': roc_auc_score(y_test, y_prob_xgb),
    'precision': precision_score(y_test, y_pred_xgb, pos_label=1),
    'recall': recall_score(y_test, y_pred_xgb, pos_label=1)
}

xgb_results

{'accuracy': 0.6129225246784326,
 'f1': 0.497085114652157,
 'roc_auc': 0.6432048299228437,
 'precision': 0.43936791480590864,
 'recall': 0.5722595078299776}

## Paso 4: Evaluación y Selección del Modelo

In [61]:

# Crear un diccionario con todos los resultados
results_dict = {
    'Logistic Regression': logreg_results,
    'Decision Tree': tree_results,
    'Random Forest': rf_results,
    'XGBoost': xgb_results
}

# Convertirlo en DataFrame
df_results = pd.DataFrame(results_dict).T

# Ordenar columnas
df_results = df_results[['accuracy', 'f1', 'roc_auc', 'precision', 'recall']]

df_results.style.highlight_max(color='green')

,accuracy,f1,roc_auc,precision,recall
Logistic Regression,0.678433,0.223827,0.653924,0.579439,0.138702
Decision Tree,0.654053,0.358747,0.542401,0.471574,0.289485
Random Forest,0.621448,0.406565,0.575098,0.427094,0.387919
XGBoost,0.612923,0.497085,0.643205,0.439368,0.572260


## Paso 5: La Decisión de Negocio (El Entregable)

**¿Qué modelo elegiste como "ganador" y por qué?** 

Después de comparar los cuatro modelos utilizando métricas como F1, ROC-AUC, precision y recall, el modelo que ofrece el mejor desempeño global es **XGBoost**.

- **Mejor F1 Score (0.497)**: El F1 combina precisión y recall, por lo que es la métrica más adecuada en un problema desbalanceado. **XGBoost** supera a todos los demás modelos (F1: Decision Tree 0.358, RandomForest 0.406, Logistic 0.224).
- **Mayor Recall (0.572)**: Es el modelo que mejor identifica a las sesiones que realmente visitan la página de ofertas. Esto es crucial porque la meta del proyecto es detectar a los buscadores de ofertas. Casi duplica el recall del RandomForest y cuadruplica el de la regresión logística (0.138).

**¿Cuáles son los 3 factores más importantes?**

In [57]:
# 1. Sacar el preprocesador y el modelo ya entrenados del pipeline
preprocessor_fitted = xgb_model.named_steps['preprocessor']
xgb_clf = xgb_model.named_steps['classifier']

# 2. Obtener los nombres de las columnas de salida del preprocessor

# 2.1. Las numéricas quedan igual
numeric_features_out = numeric_features

# 2.2. Las categóricas salen expandidas por OneHotEncoder
ohe = preprocessor_fitted.named_transformers_['cat'].named_steps['onehot']
categorical_features_out = ohe.get_feature_names_out(categorical_features)

# 2.3. Concatenar nombres finales
feature_names = list(numeric_features_out) + list(categorical_features_out)

# 3. Importancias desde el modelo XGBoost
importances = xgb_clf.feature_importances_

# 4. DataFrame de importancias
feat_importances = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Top 3
top_features = feat_importances.head(3)
top_features

,feature,importance
4,distinct_products,0.236178
7,main_category_mode_3,0.170251
0,total_clicks,0.145115


**¿Cuál es tu recomendación de negocio?**

Los resultados indican que los usuarios que tienen mayor probabilidad de visitar la página de ofertas (“Sale”) muestran tres comportamientos específicos: exploran muchos productos distintos, navegan principalmente en la categoría skirts, y realizan un alto número de clics en la sesión. Con base en estos hallazgos, se recomienda implementar estrategias que aprovechen estos patrones de comportamiento para incrementar la conversión hacia la página de ofertas y mejorar la eficacia comercial del sitio. Por ejemplo:

- Activar un banner dinámico de “Ofertas” para sesiones con alta exploración
- Diseñar campañas enfocadas en usuarios interesados en la categoría ‘Skirts’
- Aprovechar la alta actividad (total_clicks) para mostrar mensajes contextuales (“¿Buscas descuentos?” o “Explora nuestras ofertas”)